In [1]:
from src.code_gen.code_generator import CCodeGenerator

generator = CCodeGenerator()

In [3]:
code = generator('''  
    function pow(x) => x^2;

    let a = [1,2,3] in {
        for (x in a) print(x);
                 
        let b = 0 in 
            while(b < 2) {
                a[0] := a[0] + b;
                b := b + 1;
            };
                 
        let b = 0 in {
                for (x in a) b := b + x;
    
                print([pow(x) + b || x in a]);
            };
    };
    '''
, False)

print(code)

Object* function_pow (Object* p0);

Object* loopBlock0(Object* v0);

Object* condition0(Object* v2, Object* v0);

Object* loopBlock1(Object* v2, Object* v0);

Object* loopBlock2(Object* v3, Object* v0);

Object* selector0 (Object* v3, Object* v0, Object* v5);

Object* vectorComprehension0 (Object* v3, Object* v0);

Object* condition0(Object* v2, Object* v0) {
   return numberLessThan(v2, createNumber(2));
}

Object* loopBlock0(Object* v0) {
   Object* return_obj = NULL;
   Object* v1 = NULL;
   Object* vector = v0;
   Object** list = getAttributeValue(vector, "list");
   int size = *(int*)getAttributeValue(vector, "size");

   for(int i = 0; i < size; i++) {
      v1 = list[i];

      return_obj = function_print(v1);
   }
   return return_obj;
}

Object* loopBlock1(Object* v2, Object* v0) {
   Object* return_obj = NULL;
   while(*((bool*)getAttributeValue(condition0(v2, v0), "value"))) {
      replaceObject(getElementOfVector(v0, createNumber(0)), numberSum(getElementOfVector(v0, creat